# Integrating Macroeconomic Indicators into Rents and Transactions Data

## Introduction

In this chapter, I focus on enhancing the transactions and rents datasets by integrating relevant macroeconomic indicators. My goal is to ensure that these datasets are enriched with external factors that have a significant influence on the real estate market. By including these indicators, I aim to capture the broader economic and social context that affects property values and rental trends.


# Rents Dataset

In [ ]:
import pandas as pd

rents_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Rents & Transactions\rents_cleaned.xlsx"
cpi_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Consumer Price Index\Consumer_Price_Index_Monthly.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\rents_with_cpi.csv"

rents_df = pd.read_excel(rents_file_path)
cpi_df = pd.read_excel(cpi_file_path)

rents_df['YearMonth'] = pd.to_datetime(rents_df['Registration Date']).dt.to_period('M')
cpi_df['YearMonth'] = pd.to_datetime(cpi_df['Date']).dt.to_period('M')

merged_df = rents_df.merge(cpi_df[['YearMonth', 'CPI Value']], on='YearMonth', how='left')

merged_df.drop(columns='YearMonth', inplace=True)

merged_df.to_csv(output_file_path, index=False)

print("Merge complet! Fișierul rezultat a fost salvat ca 'rents_with_cpi.csv'.")

In [ ]:
import pandas as pd

rents_with_cpi_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\rents_with_cpi.xlsx"
aed_usd_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Currency Strength\AED-USD_Cleaned.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\rents_with_cpi_aed_usd.csv"

rents_with_cpi_df = pd.read_excel(rents_with_cpi_file_path)
aed_usd_df = pd.read_excel(aed_usd_file_path)

rents_with_cpi_df['Registration Date'] = pd.to_datetime(rents_with_cpi_df['Registration Date'])
aed_usd_df['Date'] = pd.to_datetime(aed_usd_df['Date'])

merged_df = rents_with_cpi_df.merge(
    aed_usd_df[['Date', 'Close', 'Return']],
    left_on='Registration Date',
    right_on='Date',
    how='left'
)

merged_df.drop(columns='Date', inplace=True)

merged_df.to_csv(output_file_path, index=False)

print("Merge complet! Fișierul rezultat a fost salvat ca 'rents_with_cpi_aed_usd.csv'.")


In [10]:
import pandas as pd

gdp_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Gross Domestic Product\GDP_Quarterly_Constant_Prices_cleaned.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\GDP_Daily_Expanded.xlsx"

gdp = pd.read_excel(gdp_file_path)

gdp['Date'] = pd.to_datetime(gdp['Date'])

daily_gdp_data = []

for i, row in gdp.iterrows():
    start_date = row['Date']
    
    if row['QUARTER'] == 'Q1':
        end_date = pd.Timestamp(f"{row['Year']}-03-31")
    elif row['QUARTER'] == 'Q2':
        end_date = pd.Timestamp(f"{row['Year']}-06-30")
    elif row['QUARTER'] == 'Q3':
        end_date = pd.Timestamp(f"{row['Year']}-09-30")
    elif row['QUARTER'] == 'Q4':
        end_date = pd.Timestamp(f"{row['Year']}-12-31")
    
    if end_date > gdp['Date'].max():
        end_date = gdp['Date'].max()
    
    daily_gdp_data.extend([
        {
            'Date': date,
            'GDP Value': row['GDP Value'],
            'GDP Category': row['GDP Category']
        }
        for date in pd.date_range(start=start_date, end=end_date, freq='D')
    ])

daily_gdp = pd.DataFrame(daily_gdp_data)

daily_gdp.to_excel(output_file_path, index=False)

print(f"Corrected daily GDP data has been saved at '{output_file_path}'.")



Corrected daily GDP data has been saved at 'D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\GDP_Daily_Expanded.xlsx'.


In [1]:
import pandas as pd

rents_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\rents_with_cpi_aed_usd.xlsx"
gdp_daily_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\GDP_Daily_Expanded.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_rents_with_tot_gdp_split.xlsx"

rents = pd.read_excel(rents_file_path)
gdp_daily = pd.read_excel(gdp_daily_file_path)

rents['Registration Date'] = pd.to_datetime(rents['Registration Date'])
gdp_daily['Date'] = pd.to_datetime(gdp_daily['Date'])

filtered_gdp = gdp_daily[gdp_daily['GDP Category'] == 'TOT_GDP']

rents.sort_values('Registration Date', inplace=True)
filtered_gdp.sort_values('Date', inplace=True)

filtered_gdp = filtered_gdp.drop_duplicates(subset=['Date'])

merged_data = pd.merge(
    rents,
    filtered_gdp,
    left_on='Registration Date',
    right_on='Date',
    how='left'
)

merged_data.drop(columns=['Date'], inplace=True)

chunk_size = 1_000_000  
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    for i in range(0, len(merged_data), chunk_size):
        sheet_name = f"Sheet_{i // chunk_size + 1}"  
        merged_data.iloc[i:i + chunk_size].to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Merged file saved with multiple sheets to: {output_file_path}")


C:\Users\User\AppData\Local\Temp\ipykernel_8652\294076567.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_gdp.sort_values('Date', inplace=True)


Merged file saved with multiple sheets to: D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_rents_with_tot_gdp_split.xlsx


In [4]:
import pandas as pd
from tqdm import tqdm

rents_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_rents_with_tot_gdp_split.xlsx"
population_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Population\Population_Estimates_and_Growth_by_Gender_cleaned.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Merged_CSV_Corrected.csv"

print("Loading datasets...")
rents = pd.read_excel(rents_file_path, sheet_name=None)  
population = pd.read_excel(population_file_path)

print("Preparing population data...")
population["Year"] = pd.to_datetime(population["Date"]).dt.year
population = population[["Year", "Value", "GENDER"]]  
population.rename(columns={"Value": "Population"}, inplace=True)

merged_data = []

print("Merging sheets...")
for sheet_name, sheet_data in tqdm(rents.items(), desc="Merging Sheets"):
    sheet_data['Year'] = pd.to_datetime(sheet_data['Registration Date']).dt.year

    sheet_data['GENDER'] = '_T'

    merged_sheet = pd.merge(
        sheet_data,
        population,
        on=["Year", "GENDER"],  
        how="left" 
    )

    merged_data.append(merged_sheet)

print("Combining all sheets into a single DataFrame...")
final_merged_data = pd.concat(merged_data, ignore_index=True)

print("Saving merged data to CSV...")
final_merged_data.to_csv(output_file_path, index=False)

print(f"Merged file with annual population data saved to: {output_file_path}")


Loading datasets...
Preparing population data...
Merging sheets...


Merging Sheets: 100%|████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.31it/s]


Combining all sheets into a single DataFrame...
Saving merged data to CSV...
Merged file with annual population data saved to: D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Merged_CSV_Corrected.csv


In [7]:
import pandas as pd

rents_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Merged_CSV_Corrected.csv"
tourism_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Tourism\Hotel_Establishments_Main_Indicators_cleaned.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Merged_With_Tourism.csv"

print("Loading datasets...")
rents = pd.read_csv(rents_file_path)
tourism = pd.read_excel(tourism_file_path)

print("Preparing tourism data...")
tourism["Year"] = pd.to_datetime(tourism["TIME_PERIOD"]).dt.year
tourism_filtered = tourism[tourism["H_INDICATOR"] == "RR"]  
tourism_prepared = tourism_filtered[["Year", "OBS_VALUE"]]  
tourism_prepared.rename(columns={"OBS_VALUE": "Revenue_Per_Room"}, inplace=True)

print("Merging datasets...")
rents_merged = pd.merge(
    rents,
    tourism_prepared,
    on="Year",  
    how="left"  
)

print("Saving merged data...")
rents_merged.to_csv(output_file_path, index=False)

print(f"Merged file with tourism data saved to: {output_file_path}")


Loading datasets...


C:\Users\User\AppData\Local\Temp\ipykernel_10012\2178394570.py:10: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  rents = pd.read_csv(rents_file_path)


Preparing tourism data...
Merging datasets...
Saving merged data...


C:\Users\User\AppData\Local\Temp\ipykernel_10012\2178394570.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tourism_prepared.rename(columns={"OBS_VALUE": "Revenue_Per_Room"}, inplace=True)


Merged file with tourism data saved to: D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Merged_With_Tourism.csv


In [15]:
import pandas as pd

rents_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Merged_With_Tourism.csv"
rents_data = pd.read_csv(rents_file_path)

indicators_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\World Development Indicators\World_Development_Indicator_cleaned.xlsx"
indicators_data = pd.read_excel(indicators_file_path)

rents_data['Year'] = pd.to_numeric(rents_data['Year'], errors='coerce')
indicators_data['Year'] = pd.to_datetime(indicators_data['Year']).dt.year

selected_indicators = [
    'Urban population (% of total population)',
    'Population ages 15-64 (% of total population)',
    'GDP per capita (current US$)',
    'Unemployment, total (% of total labor force) (national estimate)',
    'Population in urban agglomerations of more than 1 million (% of total population)'
]
indicators_data_filtered = indicators_data[indicators_data['Indicator Name'].isin(selected_indicators)]

indicators_pivot = indicators_data_filtered.pivot_table(
    index=['Country Code', 'Year'],
    columns='Indicator Name',
    values='Value'
).reset_index()

merged_data = pd.merge(
    rents_data,
    indicators_pivot,
    how='left',
    left_on='Year',
    right_on='Year'
)

output_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Merged_With_Indicators.csv"
merged_data.to_csv(output_path, index=False)

print(f"Merged file saved at: {output_path}")


C:\Users\User\AppData\Local\Temp\ipykernel_10012\457750714.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  rents_data = pd.read_csv(rents_file_path)


Merged file saved at: D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Merged_With_Indicators.csv


## Integrating Macroeconomic Indicators into the Rents Dataset

### Objective

This chapter focuses on enhancing the **rents dataset** with essential macroeconomic indicators. The goal is to provide a holistic view of the economic context that influences rental trends in Dubai. Each data integration was carefully chosen and implemented to ensure that the dataset captures the multidimensional factors driving the real estate market. Below, I detail the steps, the reasoning behind each decision, and the categories selected.

---

### Steps and Rationale

#### 1. Adding Consumer Price Index (CPI)
- **Why CPI?**  
  Inflation directly affects the purchasing power of consumers and investors, influencing rental prices. The **CPI** is a widely used economic indicator that reflects the cost of living over time.
- **Approach Taken:**  
  - I extracted the `YearMonth` (e.g., "2022-03") from the `Registration Date` in the rents dataset and matched it with the same column in the CPI data.
  - The left join ensured that all rent transactions were retained, even if some months lacked CPI data.
  - The `CPI Value` column added inflation trends to the dataset.

#### 2. Integrating AED-USD Exchange Rates
- **Why Currency Strength?**  
  Dubai’s real estate market attracts a significant number of international investors. Fluctuations in the AED-USD exchange rate can impact their purchasing decisions and, consequently, rental demand.
- **Approach Taken:**  
  - Daily exchange rates were aligned with the `Registration Date` in the rents dataset.
  - I chose the `Close` rate to reflect the finalized daily exchange rate and `Return` to capture the rate's volatility.
  - This decision provides insights into how currency strength correlates with rental behavior, especially during periods of significant fluctuation.

#### 3. Incorporating Daily GDP Data
- **Why GDP?**  
  Gross Domestic Product (GDP) is a comprehensive measure of economic performance. By integrating GDP data, I aimed to evaluate the correlation between rental prices and overall economic activity.
- **Approach Taken:**  
  - I expanded quarterly GDP data into daily values to achieve a granular match with the `Registration Date` in the rents dataset.
  - Only the **Total GDP (TOT_GDP)** category was retained as it reflects the overall economic health, making it more relevant than subcategories like sectoral GDP.
  - This provided a detailed, daily view of how economic performance may have influenced rental patterns.

#### 4. Adding Population Data
- **Why Population Data?**  
  Population trends are fundamental to understanding housing demand. Factors such as population growth, migration, and demographic distribution can directly affect the rental market.
- **Approach Taken:**  
  - I filtered the **Population Estimates and Growth by Gender** dataset to focus on the `_T` (total) category to capture overall population trends.
  - Using the `Year` from the `Registration Date`, I merged the annual population data with the rents dataset.
  - This decision enables analysis of how demographic changes influence rental behavior, especially in a rapidly growing city like Dubai.

#### 5. Integrating Tourism Data
- **Why Tourism Indicators?**  
  Dubai is a global tourism hub, and transient populations such as tourists contribute to rental demand. Indicators like **Revenue Per Room (RR)** offer insights into market dynamics in tourist hotspots.
- **Approach Taken:**  
  - I filtered the tourism data to retain only the `Revenue Per Room (RR)` metric, which is a direct indicator of demand and revenue in the hospitality sector.
  - The data was merged based on `Year`, aligning it with the `Registration Date` in the rents dataset.
  - This inclusion allows for an understanding of how seasonal and annual tourism trends impact rental prices.

#### 6. Adding World Development Indicators
- **Why World Development Indicators?**  
  Global and regional development indicators provide additional layers of context to analyze the rental market. For example:
  - **Urban population (%)** reflects urbanization levels.
  - **Population ages 15-64 (%)** captures the working-age population.
  - **GDP per capita** indicates individual economic capacity.
  - **Unemployment rates** reflect labor market health.
  - **Population in urban agglomerations of >1 million** highlights city-level population density.
- **Approach Taken:**  
  - I selected only relevant indicators based on their potential impact on the rental market.
  - The data was filtered and pivoted to align with the rents dataset on the `Year`.
  - This integration helps examine long-term socioeconomic trends influencing rental prices.

---

### Validation and Challenges

#### Validation Process
- **Row Count Consistency:**  
  I ensured that no rows were lost or duplicated during the merging process.
- **Missing Data Analysis:**  
  I analyzed missing values for each added column and noted their proportions for future imputation or exclusion.
- **Data Distribution:**  
  I examined the statistical distribution of each indicator to verify alignment with expected trends.
- **Unmatched Records:**  
  Unmatched records were identified and attributed to date ranges outside the scope of the external datasets.

#### Challenges
1. **Handling Missing Data:**  
   Some indicators, such as unemployment rates, were incomplete for specific years. I decided to retain these rows and address missing data during preprocessing.
2. **Temporal Alignment:**  
   Aligning datasets with different temporal granularities (e.g., daily vs. annual) required careful handling to avoid mismatches.
3. **Indicator Relevance:**  
   I had to carefully filter and select only indicators that provided meaningful insights into rental market trends.

---

### Outcome

The enriched dataset now includes the following macroeconomic indicators:
- **CPI Value** for inflation trends.
- **AED-USD exchange rates** for currency fluctuations.
- **Daily GDP values** for economic activity.
- **Population estimates** for demographic analysis.
- **Tourism indicators** for external demand insights.
- **World Development Indicators** for socioeconomic context.

This comprehensive dataset offers a robust foundation for analyzing the impact of macroeconomic factors on Dubai’s rental market. It is ready for exploratory data analysis, correlation studies, and predictive modeling.

---

### Next Steps
1. Perform exploratory data analysis (EDA) to uncover trends and relationships between the added indicators and rental prices.
2. Develop machine learning models to predict rental prices using this enriched dataset.
3. Identify actionable insights and trends that can guide strategic decisions for investors and policymakers.


# Transactions Dataset

In [6]:
import pandas as pd

transactions_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Rents & Transactions\transactions_cleaned.xlsx"
cpi_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Consumer Price Index\Consumer_Price_Index_Monthly.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\transactions_with_cpi.csv"

transactions_df = pd.read_excel(transactions_file_path)
cpi_df = pd.read_excel(cpi_file_path)

transactions_df['YearMonth'] = pd.to_datetime(transactions_df['Transaction Date']).dt.to_period('M')
cpi_df['YearMonth'] = pd.to_datetime(cpi_df['Date']).dt.to_period('M')

cpi_df = cpi_df.groupby('YearMonth').agg({'CPI Value': 'mean'}).reset_index()

merged_df = transactions_df.merge(cpi_df, on='YearMonth', how='left')

merged_df['CPI Value'].fillna(0, inplace=True)

merged_df.drop(columns='YearMonth', inplace=True)

merged_df.to_csv(output_file_path, index=False)

print(f"Merge complet! Fișierul rezultat a fost salvat ca '{output_file_path}'.")


C:\Users\User\AppData\Local\Temp\ipykernel_13852\2434700888.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['CPI Value'].fillna(0, inplace=True)


Merge complet! Fișierul rezultat a fost salvat ca 'D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\transactions_with_cpi.csv'.


In [9]:
import pandas as pd

merged_transactions_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\transactions_with_cpi.csv"
currency_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Currency Strength\AED-USD_Cleaned.xlsx"
output_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\transactions_with_cpi_currency.csv"

merged_transactions_df = pd.read_csv(merged_transactions_path)
currency_df = pd.read_excel(currency_path)

merged_transactions_df['Transaction Date'] = pd.to_datetime(merged_transactions_df['Transaction Date'])
currency_df['Date'] = pd.to_datetime(currency_df['Date'])

currency_relevant = currency_df[['Date', 'Adj Close', 'Return']]

merged_with_currency = merged_transactions_df.merge(currency_relevant, left_on='Transaction Date', right_on='Date', how='left')

merged_with_currency.drop(columns=['Date'], inplace=True)

merged_with_currency.to_csv(output_path, index=False)

print(f"Merge complete! The resulting file has been saved at '{output_path}'.")


C:\Users\User\AppData\Local\Temp\ipykernel_13852\3992648669.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_transactions_df = pd.read_csv(merged_transactions_path)


Merge complete! The resulting file has been saved at 'D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\transactions_with_cpi_currency.csv'.


In [12]:
import pandas as pd

transactions_with_currency_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\transactions_with_cpi_currency.csv"
gdp_daily_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Gross Domestic Product\GDP_Daily_Expanded.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_transactions_with_gdp.csv"

transactions = pd.read_csv(transactions_with_currency_path)
gdp_daily = pd.read_excel(gdp_daily_path)

transactions['Transaction Date'] = pd.to_datetime(transactions['Transaction Date'])
gdp_daily['Date'] = pd.to_datetime(gdp_daily['Date'])

filtered_gdp = gdp_daily[gdp_daily['GDP Category'] == 'TOT_GDP']

transactions.sort_values('Transaction Date', inplace=True)
filtered_gdp.sort_values('Date', inplace=True)

filtered_gdp = filtered_gdp.drop_duplicates(subset=['Date'])

merged_data = pd.merge(
    transactions,
    filtered_gdp,
    left_on='Transaction Date',
    right_on='Date',
    how='left'
)

merged_data.drop(columns=['Date'], inplace=True)

merged_data.to_csv(output_file_path, index=False)
print(f"Merged file saved to: {output_file_path}")


C:\Users\User\AppData\Local\Temp\ipykernel_13852\3551862849.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv(transactions_with_currency_path)
C:\Users\User\AppData\Local\Temp\ipykernel_13852\3551862849.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_gdp.sort_values('Date', inplace=True)


Merged file saved to: D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_transactions_with_gdp.csv


In [22]:
import pandas as pd
from tqdm import tqdm  

transactions_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_transactions_with_gdp.csv"
population_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Population\Population_Estimates_and_Growth_by_Gender_cleaned.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_transactions_with_total_population.csv"

print("Loading datasets...")
transactions = pd.read_csv(transactions_file_path)
population = pd.read_excel(population_file_path)
print("Datasets loaded successfully.")

print("Filtering population data...")
population["Year"] = pd.to_datetime(population["Date"]).dt.year  
population_total = population[population["GENDER"] == "_T"]  
population_total = population_total[["Year", "Value"]]  
population_total.rename(columns={"Value": "Total Population"}, inplace=True)
print("Population data filtered.")

transactions["Year"] = pd.to_datetime(transactions["Transaction Date"]).dt.year

print("Merging transactions with population data...")
merged_data = pd.merge(
    transactions,
    population_total,
    on="Year",  
    how="left" 
)
print("Merging completed.")

print("Saving merged data...")
merged_data.to_csv(output_file_path, index=False)
print(f"Merged file with total population data saved to: {output_file_path}")


Loading datasets...


C:\Users\User\AppData\Local\Temp\ipykernel_13852\1424244179.py:11: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv(transactions_file_path)


Datasets loaded successfully.
Filtering population data...
Population data filtered.
Merging transactions with population data...
Merging completed.
Saving merged data...
Merged file with total population data saved to: D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_transactions_with_total_population.csv


In [25]:
import pandas as pd

transactions_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_transactions_with_total_population.csv"
tourism_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Tourism\Hotel_Establishments_Main_Indicators_cleaned.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_transactions_with_tourism.csv"

print("Loading datasets...")
transactions = pd.read_csv(transactions_file_path)
tourism = pd.read_excel(tourism_file_path)

print("Preparing tourism data...")
tourism["Year"] = pd.to_datetime(tourism["TIME_PERIOD"]).dt.year 
tourism_filtered = tourism[tourism["H_INDICATOR"] == "RR"] 
tourism_prepared = tourism_filtered[["Year", "OBS_VALUE"]] 
tourism_prepared.rename(columns={"OBS_VALUE": "Revenue_Per_Room"}, inplace=True)

print("Merging datasets...")
merged_data = pd.merge(
    transactions,
    tourism_prepared,
    on="Year", 
    how="left" 
)

print("Saving merged data...")
merged_data.to_csv(output_file_path, index=False)

print(f"Merged file with tourism data saved to: {output_file_path}")


Loading datasets...


C:\Users\User\AppData\Local\Temp\ipykernel_13852\2682622903.py:10: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv(transactions_file_path)


Preparing tourism data...
Merging datasets...


C:\Users\User\AppData\Local\Temp\ipykernel_13852\2682622903.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tourism_prepared.rename(columns={"OBS_VALUE": "Revenue_Per_Room"}, inplace=True)


Saving merged data...
Merged file with tourism data saved to: D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_transactions_with_tourism.csv


In [27]:
import pandas as pd

rents_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\merged_transactions_with_tourism.csv"
indicators_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\World Development Indicators\World_Development_Indicator_cleaned.xlsx"
output_file_path = r"D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Transactions_Merged_With_Indicators.csv"

print("Loading datasets...")
rents_data = pd.read_csv(rents_file_path)
indicators_data = pd.read_excel(indicators_file_path)
print("Datasets loaded successfully.")

rents_data['Year'] = pd.to_numeric(rents_data['Year'], errors='coerce')
indicators_data['Year'] = pd.to_datetime(indicators_data['Year']).dt.year

print("Filtering indicators of interest...")
selected_indicators = [
    'Urban population (% of total population)',
    'Population ages 15-64 (% of total population)',
    'GDP per capita (current US$)',
    'Unemployment, total (% of total labor force) (national estimate)',
    'Population in urban agglomerations of more than 1 million (% of total population)'
]
indicators_data_filtered = indicators_data[indicators_data['Indicator Name'].isin(selected_indicators)]

print("Pivoting indicators data...")
indicators_pivot = indicators_data_filtered.pivot_table(
    index=['Year'],
    columns='Indicator Name',
    values='Value'
).reset_index()

print("Merging datasets...")
merged_data = pd.merge(
    rents_data,
    indicators_pivot,
    how='left',
    on='Year'
)

print("Saving merged data...")
merged_data.to_csv(output_file_path, index=False)

print(f"Merged file saved at: {output_file_path}")


Loading datasets...


C:\Users\User\AppData\Local\Temp\ipykernel_13852\1185864860.py:10: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  rents_data = pd.read_csv(rents_file_path)


Datasets loaded successfully.
Filtering indicators of interest...
Pivoting indicators data...
Merging datasets...
Saving merged data...
Merged file saved at: D:\Desights.ai\Dubai Real Estate Price Prediction Challenge\Cleaned Files\Transactions_Merged_With_Indicators.csv


## Enhancing the Transactions Dataset with Macroeconomic Indicators

### Objective

In this section, I document the process of enriching the **transactions dataset** with various macroeconomic indicators to gain deeper insights into the economic factors influencing property transactions in Dubai. Each added dataset provides a unique perspective on the real estate market. Below, I explain the rationale for selecting specific indicators and the steps undertaken to integrate them seamlessly into the transactions dataset.

---

### Steps and Rationale

#### 1. Adding the Consumer Price Index (CPI)
- **Why CPI?**  
  The **Consumer Price Index** reflects inflation trends, a crucial economic factor that impacts purchasing power and property values. By incorporating CPI, I can analyze how inflation correlates with transaction values over time.
- **Approach Taken:**  
  - I extracted the `YearMonth` (e.g., "2023-07") from the `Transaction Date` in the transactions dataset and aligned it with the same column in the CPI dataset.
  - Averages were calculated for duplicate months in the CPI data to ensure each `YearMonth` had a unique CPI value.
  - The `CPI Value` column was then added to the transactions dataset, ensuring that inflation trends were captured.

---

#### 2. Incorporating AED-USD Exchange Rates
- **Why Currency Strength?**  
  The AED-USD exchange rate plays a pivotal role in Dubai's property market due to the high volume of international transactions. Fluctuations in currency strength can directly impact investor behavior and property prices.
- **Approach Taken:**  
  - Exchange rates were matched with the `Transaction Date` in the transactions dataset using a left join.
  - Key columns such as `Adj Close` (daily closing exchange rate) and `Return` (daily change percentage) were added to the transactions file.
  - These additions enable me to investigate how exchange rate fluctuations influence property transactions, particularly for international buyers.

---

#### 3. Adding Daily GDP Data
- **Why GDP?**  
  Gross Domestic Product (GDP) serves as a measure of overall economic performance. By integrating GDP data, I can assess the relationship between economic health and transaction volumes or values.
- **Approach Taken:**  
  - I filtered the GDP data to retain only the **Total GDP (TOT_GDP)** category, as it provides a holistic measure of economic performance.
  - The quarterly GDP data was expanded into daily values to match the granularity of the `Transaction Date`.
  - A left join ensured that each transaction was associated with the corresponding GDP value on the transaction date.

---

#### 4. Incorporating Population Data
- **Why Population Data?**  
  Demographic trends influence housing demand, making population data a critical factor in real estate analysis. Population growth or decline provides insights into market dynamics.
- **Approach Taken:**  
  - I filtered the **Population Estimates and Growth by Gender** dataset to retain only the `_T` (total population) category for a comprehensive view.
  - Population data was matched with transactions based on the `Year` extracted from the `Transaction Date`.
  - The resulting dataset provides a foundation for understanding how demographic changes drive transaction behavior.

---

#### 5. Adding Tourism Indicators
- **Why Tourism Indicators?**  
  Tourism is a significant driver of Dubai’s economy and can affect property transactions, especially for properties in high-demand areas frequented by tourists.
- **Approach Taken:**  
  - I selected the **Revenue Per Room (RR)** indicator from the tourism dataset, which reflects the economic performance of the hospitality sector.
  - The data was merged with transactions based on the `Year` extracted from the `Transaction Date`.
  - This allows for analysis of how fluctuations in tourism-related revenue correlate with property transaction trends.

---

#### 6. Incorporating World Development Indicators (WDI)
- **Why World Development Indicators?**  
  These indicators provide a broader socioeconomic context, helping to understand long-term market drivers. Key indicators include:
  - **Urban population (% of total population):** Indicates urbanization trends.
  - **Population ages 15-64 (% of total population):** Highlights the working-age population.
  - **GDP per capita:** Reflects economic capacity at an individual level.
  - **Unemployment rates:** Indicates labor market health.
  - **Population in urban agglomerations of >1 million:** Captures population density in metropolitan areas.
- **Approach Taken:**  
  - Relevant indicators were filtered and pivoted for easy integration.
  - A left join was performed based on the `Year` extracted from the `Transaction Date`.
  - These additions provide insights into how broader socioeconomic factors influence property transactions.

---

### Validation and Challenges

#### Validation Process
1. **Row Count Consistency:**  
   The merged dataset was verified to ensure no rows were lost during integration.
2. **Missing Data Analysis:**  
   I evaluated the proportion of missing values for each added indicator and identified potential reasons, such as temporal mismatches or data unavailability.
3. **Indicator Distribution:**  
   The distribution of each added indicator was analyzed to ensure alignment with expected trends.

#### Challenges
- **Temporal Alignment:**  
  Datasets with different temporal granularities (e.g., monthly CPI vs. daily transactions) required careful preprocessing to ensure accurate alignment.
- **Incomplete Data:**  
  Missing values in certain indicators (e.g., tourism data for earlier years) necessitate further imputation or exclusion in subsequent analyses.

---

### Outcome

The enhanced transactions dataset now includes:
- **CPI Value** for inflation trends.
- **AED-USD exchange rates** for currency fluctuations.
- **Daily GDP values** for economic activity.
- **Population estimates** for demographic context.
- **Tourism indicators** for demand insights.
- **World Development Indicators** for socioeconomic analysis.

This enriched dataset is ready for exploratory data analysis and predictive modeling, providing a comprehensive view of the factors influencing Dubai's property market.

---

### Next Steps
1. Perform exploratory data analysis (EDA) to uncover patterns and correlations between the added indicators and transaction values.
2. Build machine learning models to predict property transaction prices using the enriched dataset.
3. Derive actionable insights for investors, policymakers, and stakeholders based on the findings.
